# `swift_too` module

## Swift_GUANO example - querying GUANO data, i.e. dumps of BAT event data

### API version = 1.2, `swifttools` = 3.0.6

#### Author: Jamie A. Kennea (Penn State)

What is GUANO? It is the Gamma-ray Urgent Archive for Novel Opportunities. GUANO is fully explained in the paper by [Tohuvavohu et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...900...35T/abstract). Essentially it is this, the Swift Burst Alert Telescope (BAT) is a coded mask telesope, which records individual X-ray photons as events. These events are then processed onboard to look for GRB triggers. However, sometimes the onboard trigger algorithm fails to find a GRB that is present in the BAT field of view. In these cases, it would be best to look at the event data on the ground to do a more detailed analysis to look for GRB events, however, because of telemetery limitations, the event data is only held onboard for around 25-30 mins before it is deleted. 

The GUANO program seeks to solve this problem, but proactively commanding Swift to keep and downlink BAT event data for times of interest. Right now this includes times at which other instruments trigger, such as Fermi/GBM, INTEGRAL, CALET and other GRB missions, as well as multi-messenger observatories such as IceCube and LIGO/Virgo/Kagra. 

The `Swift_GUANO` API allows you to query when these dumps occurred and were successful, and where to look for the BAT event data that were dump. We're going to use the shorthand for this class, simply `GUANO`.


In [1]:
from swifttools.swift_too import GUANO

First a simple example, let's look for the most recent 10 successful GUANO dumps. 

In [2]:
guano = GUANO(limit=10)

In a Jupyter Notebook, we can just type the object name now to see the results, like so:

In [3]:
guano

GECAM GRB,2022-09-23 03:02:56.650000,50,200,00049835006
GECAM GRB,2022-09-22 06:12:47.650000,50,200,00059152049
GBM GRB,2022-09-21 23:25:29.370000,50,200,00021519001
GBM GRB,2022-09-19 02:26:56.010000,50,200,00015327001
GBM GRB,2022-09-18 16:13:25.760000,50,200,00015231056
GBM GRB,2022-09-18 13:00:33.190000,50,200,03111325001
Neutrino,2022-09-18 12:46:05,0,90,00014925015
GBM GRB,2022-09-17 20:35:02.610000,50,200,00075276027
GBM GRB,2022-09-17 14:44:25.600000,50,200,00059072116
GBM GRB,2022-09-17 09:04:30.470000,50,200,00096723001


Basic information given here is the source of the trigger, usually a GRB detector, but also FRB and GW detectors. The trigger time is given, along with an offset and window duration. The offset is how much the window start is biased towards the trigger time. So for offset zero, and window duration of 200s, the trigger will be in the center of the window, at 100s. For offset 50, the window is moved 50 later, so the trigger will be at 50s and the window ends at T+150s. 

Note if the Window Duration has a number in brackets after it, this means that the actual length of the dump is different from that requested. This can happen due to either the dump skimming the South Atlantic Anomaly (where no data is collected), or sometimes if the dump is commanded too late, and some of the data have been lost. Alternatively this can be due to the fact that the data has not been fully processed yet, so if the dump is recent, this may resolve itself in a few hours. 

In some cases where the number is larger than the requested duration, this can mean that other nearby trigger was merged to create a larger dump, or in extreme cases, that the dump time overlaps a BAT detected GRB. The latter case is rare as we try to filter these out on the ground.

Finally the Observation ID shows us where the BAT event data lives. If you go the Swift archive or quick-look site (recent data), e.g. here: https://swift.gsfc.nasa.gov/sdc/ql/

And look for the given observation ID, you should be able to find the BAT event data for that GUANO dump. 

### Looking at an individual GUANO dump

Let's look at an individual GUANO dump, to see what information we have.

In [4]:
guano[3]

triggertime,2022-09-19 02:26:56.010000
triggertype,GBM GRB
offset,50
duration,200
quadsaway,10526284
obsnum,00015327001
data,200.1s of BAT event data


So here we see more detail on the event itself. In addition to what we see in the summary table, there are a few more parameters. `quadsaway` is perhaps the most complicated to understand, but perhaps also not necessary. Essentially the BAT event data live in a ring buffer, this ring buffer overwrites itself, with events living around ~25 mins in the buffer before they are overwritten. `quadsaway` gives the number of quadbytes between the dump and the buffer pointer at the time of dump. If this value is positive, it means that no data was overwritten. If the value is negative, it means that some data will have been lost, this should be seen in the difference between the requested dump `duration`, and the number of seconds of BAT event data in `data`.

Also if the trigger associated with the GUANO dump had an RA/Dec, these are given. 

### Looking at the data

The final parameter, `data` has more detail, so lets look at that.

In [5]:
guano[3].data

obsid,00015327001
triggertime,2022-09-19 02:26:56.010000
begin,2022-09-19 02:26:07.115789
end,2022-09-19 02:29:27.194675
exposure,200.078886
filenames,sw00015327001bevshsl_uf.evtsw00015327001bevshpo_uf.evt
gti,2022-09-19 02:26:07.115789 - 2022-09-19 02:29:27.194675 (0:03:20)
all_gtis,2022-09-19 02:26:07.115789 - 2022-09-19 02:28:30.516079 (0:02:23)2022-09-19 02:28:30.516279 - 2022-09-19 02:29:27.194675 (0:00:56)
acs,mixed
utcf,-29.884559631347656
subthresh,False


So this is essentially metadata from a simple processing the BAT event data. Important information here, `begin` and `end` give the start and end of the event data. `exposure` is the amount of event data in seconds. `filenames` is the names of the BAT event files processed for this trigger. The different filenames represent the state of swift while the data was being taken, which is encoded into the final characters of the filename. 

A simple breakdown of the filename:

- "sw" for Swift
- The 10 character long obsid in SDC format
- "bev" = BAT event data
- "sl"/"po" or sometimes "sp", this indicates if Swift was slewing ("sl"), pointing ("po") or in some rare cases, both ("sp") for the data in that file.
- "_uf.evt" = this is unfiltered event data

If you see more than one filename, then this means that the event data for this GUANO dump is spread across multiple files, typically because Swift was slewing at some point during the dump. Note that the parameter `acs` shows if the data is slewing, pointed or mixed.

Next `gti` is an object that defines the Good Time Interval (GTI) of the GUANO dump. In this table, it is show and `begin` and `end` times of the GTI with `exposure` in brackets.

`gti` is a merged GTI, merging all the different GTIs given in `all_gtis` if they overlap. Typically `all_gtis` shows 2 GTIs if there are two `filenames`. However if there are gaps in the data, `all_gtis` can show more GTIs.

`gti` importantly only covers the continuous GTI that includes `triggertime`. Let's take a look at that.


In [6]:
guano[0].data.gti

begin,2022-09-23 03:02:08.092101
end,2022-09-23 03:05:28.191888
exposure,0:03:20
utcf,-29.908180236816406
acs,pointing
filename,sw00049835006bevshpo_uf.evt


So here we see the usual information, `begin`, `end` and `exposure`. In addition `acs` for this GTI is reported, along with the filename(s) associated with the BAT event data processed.

### Note on UT Correction Factor 
Also you'll note `utcf`, which is a negative number of seconds. This is the Univeral Time Correction Factor (UTCF), which corrects time that Swift reports to actual UT. This includes both correction for drift in the Swift clock, and leap seconds. GUANO trigger times and `begin` and `end` times are corrected using this `utcf` value, as it is important that the reported times match a common time system when comparing with other missions. 

Other times, such as those reported by Swift_ObsQuery for when Swift observations occur, are not at this time corrected to UTC, as the absolute timing for these is are not critical. UTCF corrections are typically performed during data analysis, for example when performing barycentric correction using the standard HEAsoft `barycorr` tool.

Swift times can also be corrected to UT using the HEAsoft `swifttime` analysis tool. If you use this, make sure that your Swift calibration files are upto date as the clock correction is updated regularly. 

#### Update for `swifttools` 2.4

As noted elsewhere, clock correction can now be performed on results from a class using the `clock_correct` method.

As of `swifttools` 2.4, all recorded dates in the `GUANO` class and its subclasses are automatically clock corrected. By default, all dates are UTC, not Swift time, and dates are stored in the `swiftdatetime` format, which is an extended version of Python's `datetime`, which supports clock correction, and MET conversion. So as an example, the trigger time for the above trigger:

In [7]:
guano[0].triggertime

685595006.558007,2022-09-23 03:03:26.558007,2022-09-23 03:02:56.650000,-29.908007


So this time is record in datetime by default, but you can access the time as MET by using the `met` attribute, or the time as recorded by the Swift clock using the `swifttime` attribute. E.g.

In [8]:
print(f"Trigger MET        = {guano[0].triggertime.met}")
print(f"Trigger Swift Time = {guano[0].triggertime.swifttime}")
print(f"Trigger UTC Time   = {guano[0].triggertime.utctime}")

Trigger MET        = 685595006.558007
Trigger Swift Time = 2022-09-23 03:03:26.558007
Trigger UTC Time   = 2022-09-23 03:02:56.650000


### Fetching GUANO data for a specific triggers or date ranges

The above example show fetching recent triggers, what about for date ranges, or specific times. Here we'll give a few examples of how to search for GUANO triggers using time searches. Firstly, let's see what GUANO dumps occrred during Jan 1st, 2022:

In [9]:
guano = GUANO(begin="2022-01-01", end="2022-01-02", limit=100)
guano

GBM GRB,2022-01-01 08:57:39,50,200,00014979001
GBM GRB,2022-01-01 07:16:20,50,200,00031306080
INTEGRAL_GRB,2022-01-01 05:11:22,0,90 (1202),01091527000


So on Jan 1st, 2022 there were three GUANO dumps, 2 Fermi/GBM detected GRBs and one INTEGRAL GRB. By default, GBM GRBs are dumped for 200s and INTEGRAL GRBs are dumped for 90s. However, in this case you can see that the actual amount of event data for the INTEGRAL GRB was 1202s. Let's look into why, by taking a look at the `Swift_GUANO_Data` object associated with that trigger.

In [10]:
guano[-1].data

obsid,01091527000
triggertime,2022-01-01 05:11:22
begin,2022-01-01 05:06:11.642350
end,2022-01-01 05:26:13.749165
exposure,1202.106814
filenames,sw01091527000bevshsp_uf.evt
gti,2022-01-01 05:06:11.642350 - 2022-01-01 05:26:13.749165 (0:20:02)
all_gtis,2022-01-01 05:06:11.642350 - 2022-01-01 05:26:13.749070 (0:20:02)2022-01-01 05:06:11.642365 - 2022-01-01 05:26:13.749165 (0:20:02)
utcf,-28.357940673828125
subthresh,False


So it looks like around 20 minutes of event data were dumped during starting more than 5 minutes before the trigger time. Why did this happen? Well, if you notice the filename, the slew type here is "sp" which means "slewing and pointing". Why not two files like we see in others - the reason is that this event data was dumped not due to a GUANO request, but because there was a BAT triggered GRB at the same time. 

In this case, GRB 220101A. Looking at GCN #31347 this GRB triggered BAT at 05:09:55UT, over a minute before the INTEGRAL trigger. However, it's likely that the INTEGRAL trigger was the same GRB. 

### Fetching the data

OK, so what if we want to process these data? Well, a full BAT analysis tutorial isn't the aim here, but a good start is to download the data associated with this trigger. We can do this with the `Swift_Data` class. We'll set `bat = True`, the auxil data will come down, and we'll download it our Downloads directory.

In [11]:
from swifttools.swift_too import Data

data = Data(obsid=guano[-2].obsid, bat=True, outdir="~/Downloads/", clobber=True)

Let's look to see what we got. As this was the data for a GRB, it will be quite a lot as this was a BAT triggered GRB.

In [12]:
data

00031306080/auxil,SWIFT_TLE_ARCHIVE.txt.22010.15512555.gz,Auxillary TLE file
'',sw00031306080pat.fits.gz,Auxillary attitude file
'',sw00031306080pjb.par.gz,Auxillary file
'',sw00031306080pob.cat.gz,Auxillary catalog
'',sw00031306080ppr.par.gz,Auxillary file
'',sw00031306080s.mkf.gz,Auxillary file
'',sw00031306080sao.fits.gz,Auxillary file
'',sw00031306080sat.fits.gz,Auxillary attitude file
'',sw00031306080sen.hk.gz,Auxillary housekeeping
'',sw00031306080sti.fits.gz,Auxillary file
'',sw00031306080uat.fits.gz,Auxillary attitude file


You can see that the all important file `sw00031306080bevshpo_uf.evt.gz` is up there in the bat/event directory. Here is the file details for the event files:

In [13]:
data[12]

filename,sw00031306080bevshpo_uf.evt.gz
path,00031306080/bat/event
url,https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/2022_01/00031306080/bat/event/sw00031306080bevshpo_uf.evt.gz
quicklook,False
type,BAT pointed unfiltered event event file
size,14008411
localpath,/Users/myuser/Downloads/00031306080/bat/event/sw00031306080bevshpo_uf.evt.gz


You can see that now the file is downloaded, `localpath` gives the full path to where the data lives.

#### `swifttools 3.0.6` added `download` method.

Note that you can also use the `download` method to download data, as of `swifttools` version 3.0.6. This means you don't have to start a seperate Data instance to download data, but the end result is the dame. Here is an example:


In [14]:
guano[-2].download(bat=True, outdir="~/Downloads/", clobber=True)

00031306080/auxil,SWIFT_TLE_ARCHIVE.txt.22010.15512555.gz,Auxillary TLE file
'',sw00031306080pat.fits.gz,Auxillary attitude file
'',sw00031306080pjb.par.gz,Auxillary file
'',sw00031306080pob.cat.gz,Auxillary catalog
'',sw00031306080ppr.par.gz,Auxillary file
'',sw00031306080s.mkf.gz,Auxillary file
'',sw00031306080sao.fits.gz,Auxillary file
'',sw00031306080sat.fits.gz,Auxillary attitude file
'',sw00031306080sen.hk.gz,Auxillary housekeeping
'',sw00031306080sti.fits.gz,Auxillary file
'',sw00031306080uat.fits.gz,Auxillary attitude file


That's all folks!